In [1]:
print('hello')

hello


In [2]:
import requests
from bs4 import BeautifulSoup
import re
import sqlite3
import pandas as pd
import time

In [3]:
r = requests.get("https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930")
bs_rt = BeautifulSoup(r.text, 'html.parser')

In [4]:
find_table = bs_rt.find('table', id='cTB11')

In [5]:
find_table

<table class="gHead" id="cTB11" summary="기업의 기본적인 시세정보(주가/전일대비/수익률,52주최고/최저,액면가,거래량/거래대금,시가총액,유동주식비율,외국인지분율,52주베타,수익률(1M/3M/6M/1Y))를 제공합니다.">
<caption class="blind">시세정보</caption>
<colgroup>
<col width="125"/>
<col width="*"/>
</colgroup>
<tbody>
<tr>
<th class="txt" scope="row">주가/전일대비/수익률</th>
<td class="num"><strong>
															56,000</strong>원 /
														<span class="cBk">0원</span> / <span class="cBk">0%</span>
</td>
</tr>
<tr>
<th class="txt" scope="row">52Weeks 최고/최저</th>
<td class="num">
															57,300원 /
														36,850원
														</td>
</tr>
<tr>
<th class="txt" scope="row">액면가</th>
<td class="num">
															100원
														</td>
</tr>
<tr>
<th class="txt" scope="row">거래량/거래대금</th>
<td class="num">
															12,095,500주 /
														6,775억원
														</td>
</tr>
<tr>
<th class="txt" scope="row">시가총액</th>
<td class="num">
															3,343,078억원
														</td>
</tr>
<tr>
<th class="txt" scope="row">52주베타</th>
<td class="num"

In [6]:
tr_data = find_table.findAll("tr")

In [7]:
for idx, x in enumerate(tr_data):
    print(idx, '------->', x.find("td").text.strip())

0 -------> 56,000원 /
														0원 / 0%
1 -------> 57,300원 /
														36,850원
2 -------> 100원
3 -------> 12,095,500주 /
														6,775억원
4 -------> 3,343,078억원
5 -------> 1.58
6 -------> 5,969,782,550주 / 74.55%
7 -------> 57.15%
8 -------> +7.69%/ +13.82%/ +23.08%/ +44.89%


In [8]:
x.find("td").text.strip().split('/')

['+7.69%', ' +13.82%', ' +23.08%', ' +44.89%']

In [9]:
day_pattern = re.compile("[0-9]{4}\.[0-9]{1,2}\.[0-9]{1,2}")

In [10]:
day_text = bs_rt.find('dd', class_ = "header-table-cell unit").p.text

In [11]:
day_pattern.search(day_text).group()

'2019.12.20'

In [12]:
day_text

'[기준:2019.12.20]'

In [13]:
pd.read_html(str(find_table))[0]

,0,1
0,주가/전일대비/수익률,"56,000원 / 0원 / 0%"
1,52Weeks 최고/최저,"57,300원 / 36,850원"
2,액면가,100원
3,거래량/거래대금,"12,095,500주 / 6,775억원"
4,시가총액,"3,343,078억원"
5,52주베타,1.58
6,발행주식수/유동비율,"5,969,782,550주 / 74.55%"
7,외국인지분율,57.15%
8,수익률 (1M/3M/6M/1Y),+7.69%/ +13.82%/ +23.08%/ +44.89%


In [14]:
stock_data = pd.read_csv('stock_data.csv')

In [15]:
stock_data

,순위,종목코드,종목명,현재가,대비,등락률,거래량,거래대금,시가,고가,저가,시가총액,시가총액비중(%),상장주식수,외국인 보유주식수,외국인 지분율(%),총카운트
0,1,005930,삼성전자,"56,000",0,0.0,"12,095,519","677,520,520,600","56,100","56,500","55,600","334,307,822,800,000",19.42,"5,969,782,550",NaN,NaN,2467.0
1,2,000660,SK하이닉스,"95,000","1,400",1.5,"2,791,176","264,091,139,500","94,100","95,100","93,700","69,160,224,675,000",4.02,"728,002,365",NaN,NaN,NaN
2,3,005935,삼성전자우,"45,150",-600,-1.3,"2,658,532","120,554,619,300","45,650","45,800","45,000","37,153,334,505,000",2.16,"822,886,700",NaN,NaN,NaN
3,4,035420,NAVER,"183,500","4,500",2.5,"560,761","103,004,839,500","180,000","187,000","179,500","30,243,257,982,500",1.76,"164,813,395",NaN,NaN,NaN
4,5,005380,현대차,"124,000",500,0.4,"1,001,899","123,690,538,000","124,000","124,000","122,500","26,494,855,188,000",1.54,"213,668,187",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,2463,224020,에스케이씨에스,"1,010",0,0.0,"1,053","1,005,850",955,"1,010",955,"2,727,000,000",0.00,"2,700,000",NaN,NaN,NaN
2463,2464,276240,엘리비젼,516,-2,-0.4,1,516,516,516,516,"2,669,990,400",0.00,"5,174,400",NaN,NaN,NaN
2464,2465,001529,동양3우B,"28,450",-700,-2.4,"2,199","62,132,050","28,650","29,500","28,000","2,552,590,900",0.00,"89,722",NaN,NaN,NaN
2465,2466,238500,로보쓰리,449,-79,-15.0,"14,923","6,957,429",502,502,449,"2,102,292,534",0.00,"4,682,166",NaN,NaN,NaN


In [16]:
url = "https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={}"

In [17]:
total_sise = []
pattern = re.compile("[0-9,.\+-]+")
day_pattern = re.compile("[0-9]{4}\.[0-9]{1,2}\.[0-9]{1,2}")
for i in range(200):
    code = stock_data['종목코드'][i]
    print(code)
    while(True):
        try:
            r = requests.get(url.format(code))
            break
        except Exception as e:
            print(e)
            print(r.url)
            time.sleep(3)
    bs_rt = BeautifulSoup(r.text, 'lxml')
    try:
        tr_data = bs_rt.find("table", id='cTB11').findAll("tr")
        day = day_pattern.search(bs_rt.find('dd', class_ = "header-table-cell unit").p.text).group()
        sise = []
        for idx, x in enumerate(tr_data):
            sise.extend(pattern.findall(x.find("td").text))
        if(len(sise) != 17):
            print(code)
            print('17개가 아니다')
            break
        sise.insert(0, code)
        sise.insert(1, day)
        total_sise.append(sise)
    except Exception as e:
        print(stock_data['종목명'][i])

005930
000660
005935
035420
005380
207940
012330
068270
051910
055550
005490
028260
105560
051900
017670
034730
000270
015760
006400
018260
032830
096770
033780
035720
003550
000810
066570
036570
090430
086790
010950
009150
009540
316140
010130
011170
091990
251270
030200
024110
002790
021240
035250
032640
018880
034220
267250
086280
006800
078930
000720
029780
010140
028300
004020
161390
271560
004990
088980
071050
023530
005830
028050
139480
005940
097950
012750
241560
036460
000120
016360
128940
035760
008770
047810
081660
011070
000210
009830
000100
005387
007070
042660
003670
003410
004170
030000
282330
001040
003490
138930
006360
028670
001450
008930
002380
008560
079440
263750
047050
011780
012510
180640
253450
000060
088350
036490
032500
145020
007310
000080
069960
047040
017800
000880
011790
020150
005385
012450
051600
010620
084990
026960
009420
034230
039490
086900
004800
138040
007700
240810
204320
010120
051905
056190
031430
006260
111770
010060
069620
001740
006280
009240

In [18]:
sise = pd.DataFrame(total_sise, columns=['종목코드', '기준일', '주가', '전일대비', '수익률', '최고', '최저', '액면가', '거래량', '거래대금', '시가총액',
                                  '52주베타', '발행주식수', '유동비율', '외국인지분율', '1M', '3M', '6M', '1Y'])

In [19]:
sise

,종목코드,기준일,주가,전일대비,수익률,최고,최저,액면가,거래량,거래대금,시가총액,52주베타,발행주식수,유동비율,외국인지분율,1M,3M,6M,1Y
0,005930,2019.12.20,"56,000",0,0,"57,300","36,850",100,"12,095,500","6,775","3,343,078",1.58,"5,969,782,550",74.55,57.15,+7.69,+13.82,+23.08,+44.89
1,000660,2019.12.20,"95,000","+1,400",+1.50,"96,400","56,700","5,000","2,791,180","2,641","691,602",1.71,"728,002,365",73.63,50.79,+14.87,+15.57,+41.16,+62.12
2,005935,2019.12.20,"56,000",0,0,"57,300","36,850",100,"12,095,500","6,775","3,343,078",1.58,"5,969,782,550",74.55,57.15,+7.69,+13.82,+23.08,+44.89
3,035420,2019.12.20,"183,500","+4,500",+2.51,"187,000","106,500",100,"560,761","1,030","302,433",0.29,"164,813,395",77.15,58.51,+8.26,+16.88,+64.57,+44.49
4,005380,2019.12.20,"124,000",+500,+0.40,"143,500","113,000","5,000","1,001,900","1,237","264,949",0.62,"213,668,187",65.36,40.73,-0.40,-4.25,-11.74,+2.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,170900,2019.12.20,"114,000","-2,000",-1.72,"117,000","77,500","5,000","15,263",17,"9,626",0.77,"8,443,868",56.04,25.09,+11.76,+35.55,+9.62,+8.57
196,010780,2019.12.20,"30,950",-100,-0.32,"39,550","28,100",500,"47,668",15,"9,561",1.18,"30,892,606",45.06,8.19,-7.20,0,-20.23,-0.80
197,185750,2019.12.20,"91,800",0,0,"111,000","78,000","2,500","17,781",16,"9,520",0.81,"10,370,290",62.67,12.61,-0.76,+6.99,-4.28,-8.74
198,280360,2019.12.20,"147,500","-2,000",-1.34,"201,000","139,000",500,487,1,"9,465",0.18,"6,416,717",20.83,17.15,-1.67,-1.34,-15.23,-9.51


In [28]:
query = """
        CREATE TABLE IF NOT EXISTS "stock_extend_information"(
        "종목코드" TEXT NOT NULL,
        "기준일" DATE NOT NULL,
        "주가" INT NOT NULL,
        "전일대비" TEXT NOT NULL,
        "수익률" TEXT NOT NULL,
        "최고" INT NOT NULL,
        "최저" INT NOT NULL,
        "액면가" INT NOT NULL,
        "거래량" INT NOT NULL,
        "거래대금" INT NOT NULL,
        "시가총액" BIGINT NOT NULL,
        "52주베타" FLOAT NOT NULL,
        "발행주식수" BIGINT NOT NULL,
        "유동비율" FLOAT NOT NULL,
        "외국인지분율" FLOAT NOT NULL,
        "1M" TEXT NOT NULL,
        "3M" TEXT NOT NULL,
        "6M" TEXT NOT NULL,
        "1Y" TEXT NOT NULL,
        PRIMARY KEY('종목코드')
        )
        """

In [29]:
con = sqlite3.connect("./common_stock.db")

In [30]:
cur = con.cursor()

In [31]:
cur.execute(query)

In [32]:
sise.to_sql("stock_extend_information", con=con, if_exists='append', index=False)

In [33]:
df = pd.read_sql("select * from stock_extend_information", con=con)

In [34]:
df

,종목코드,기준일,주가,전일대비,수익률,최고,최저,액면가,거래량,거래대금,시가총액,52주베타,발행주식수,유동비율,외국인지분율,1M,3M,6M,1Y
0,005930,2019.12.20,"56,000",0,0,"57,300","36,850",100,"12,095,500","6,775","3,343,078",1.58,"5,969,782,550",74.55,57.15,+7.69,+13.82,+23.08,+44.89
1,000660,2019.12.20,"95,000","+1,400",+1.50,"96,400","56,700","5,000","2,791,180","2,641","691,602",1.71,"728,002,365",73.63,50.79,+14.87,+15.57,+41.16,+62.12
2,005935,2019.12.20,"56,000",0,0,"57,300","36,850",100,"12,095,500","6,775","3,343,078",1.58,"5,969,782,550",74.55,57.15,+7.69,+13.82,+23.08,+44.89
3,035420,2019.12.20,"183,500","+4,500",+2.51,"187,000","106,500",100,"560,761","1,030","302,433",0.29,"164,813,395",77.15,58.51,+8.26,+16.88,+64.57,+44.49
4,005380,2019.12.20,"124,000",+500,+0.40,"143,500","113,000","5,000","1,001,900","1,237","264,949",0.62,"213,668,187",65.36,40.73,-0.40,-4.25,-11.74,+2.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,170900,2019.12.20,"114,000","-2,000",-1.72,"117,000","77,500","5,000","15,263",17,"9,626",0.77,"8,443,868",56.04,25.09,+11.76,+35.55,+9.62,+8.57
196,010780,2019.12.20,"30,950",-100,-0.32,"39,550","28,100",500,"47,668",15,"9,561",1.18,"30,892,606",45.06,8.19,-7.20,0,-20.23,-0.80
197,185750,2019.12.20,"91,800",0,0,"111,000","78,000","2,500","17,781",16,"9,520",0.81,"10,370,290",62.67,12.61,-0.76,+6.99,-4.28,-8.74
198,280360,2019.12.20,"147,500","-2,000",-1.34,"201,000","139,000",500,487,1,"9,465",0.18,"6,416,717",20.83,17.15,-1.67,-1.34,-15.23,-9.51


# 주가/상대수익률

In [56]:
import json
import datetime, time

In [57]:
r = requests.get("https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=001270")

In [58]:
r.text.find("encparam")

40925

In [59]:
r.text[40800:40999]

'x/cF1001.aspx\'\r\n\t\t\t\t\t, {\r\n\t\t\t\t\t\tflag: 4,\r\n\t\t\t\t\t\tcmp_cd: \'001270\',\r\n\t\t\t\t\t\tgrp_cd: grp==10?"01":"03",\r\n                        encparam: \'VHR1MGJuVkhneVJwSjd4MEF1N29aUT09\'\r\n\t\t\t\t\t}\r\n\t\t\t\t, function (res)'

In [60]:
enc_pattern = re.compile("encparam: '(.+)'")

In [61]:
enc = enc_pattern.search(r.text).group(1)

In [62]:
enc

'VHR1MGJuVkhneVJwSjd4MEF1N29aUT09'

In [63]:
payload = {}

In [64]:
payload['flag'] = 2

In [65]:
payload['cmp_cd'] = '001270'

In [66]:
payload['encparam'] = enc

In [67]:
graph_url = "https://navercomp.wisereport.co.kr/company/ajax/cF1001.aspx?"

In [68]:
r = requests.get(graph_url, params=payload)

In [69]:
r.text

'{"TRD_DT":[1514419200000.0,1517356800000.0,1519776000000.0,1522368000000.0,1525046400000.0,1527724800000.0,1530230400000.0,1532995200000.0,1535673600000.0,1538092800000.0,1540944000000.0,1543536000000.0,1545955200000.0,1548892800000.0,1551312000000.0,1553817600000.0,1556582400000.0,1559260800000.0,1561680000000.0,1564531200000.0,1567123200000.0,1569801600000.0,1572480000000.0,1574985600000.0,1576800000000.0],"D1_NM":"부국증권","D2_NM":"KOSPI대비","D3_NM":"증권대비","D1":[25150.0,28100.0,28500.0,27150.0,24350.0,23900.0,22400.0,23950.0,22850.0,22950.0,21950.0,23850.0,22450.0,20900.0,21000.0,20800.0,21400.0,20300.0,21100.0,20150.0,19250.0,20100.0,21700.0,21600.0,22000.0],"D2":[0.00,7.72,14.95,8.83,-5.12,-3.17,-5.21,2.21,-3.28,-3.71,5.02,9.85,6.55,-6.25,-5.48,-4.05,-4.22,-2.03,-2.45,-1.93,-3.21,-3.69,1.85,1.27,-1.85],"D3":[0.00,-9.25,6.45,3.43,-12.07,-12.05,-8.27,4.05,-2.25,-5.19,7.36,5.83,4.90,-9.23,-7.21,-7.40,-9.46,-10.33,-15.25,-10.69,-10.12,-9.09,1.78,-0.18,-2.10],"TRD_QTY":[5363.0,43626.0,326

In [70]:
type(json.loads(r.text))

dict

In [71]:
json.loads(r.text).keys()

dict_keys(['TRD_DT', 'D1_NM', 'D2_NM', 'D3_NM', 'D1', 'D2', 'D3', 'TRD_QTY', 'TRD_QTY_TYP'])

In [72]:
graph_1 = pd.DataFrame(json.loads(r.text))

In [73]:
graph_1

,TRD_DT,D1_NM,D2_NM,D3_NM,D1,D2,D3,TRD_QTY,TRD_QTY_TYP
0,1.514419e+12,부국증권,KOSPI대비,증권대비,25150.0,0.00,0.00,5363.0,B
1,1.517357e+12,부국증권,KOSPI대비,증권대비,28100.0,7.72,-9.25,43626.0,R
2,1.519776e+12,부국증권,KOSPI대비,증권대비,28500.0,14.95,6.45,32662.0,R
3,1.522368e+12,부국증권,KOSPI대비,증권대비,27150.0,8.83,3.43,3351.0,R
4,1.525046e+12,부국증권,KOSPI대비,증권대비,24350.0,-5.12,-12.07,4966.0,B
5,1.527725e+12,부국증권,KOSPI대비,증권대비,23900.0,-3.17,-12.05,2859.0,B
6,1.530230e+12,부국증권,KOSPI대비,증권대비,22400.0,-5.21,-8.27,1575.0,R
7,1.532995e+12,부국증권,KOSPI대비,증권대비,23950.0,2.21,4.05,1120.0,R
8,1.535674e+12,부국증권,KOSPI대비,증권대비,22850.0,-3.28,-2.25,1132.0,B
9,1.538093e+12,부국증권,KOSPI대비,증권대비,22950.0,-3.71,-5.19,794.0,B


**제공된 시간 데이터는 1970년 1월 1일 00:00:00을 기준으로 경과한 시간을 초로 환산하여 정수로 나타내는 POSIX 시간
이 시간을 우리가 보는 시간으로 변경하기 위해서 fromtimestamp() 메소드를 사용합니다.**

In [75]:
graph_1['TRD_DT'] = graph_1['TRD_DT'].apply(lambda x: datetime.datetime.fromtimestamp(int(str(x)[:10])))

In [76]:
graph_1

,TRD_DT,D1_NM,D2_NM,D3_NM,D1,D2,D3,TRD_QTY,TRD_QTY_TYP
0,2017-12-28,부국증권,KOSPI대비,증권대비,25150.0,0.00,0.00,5363.0,B
1,2018-01-31,부국증권,KOSPI대비,증권대비,28100.0,7.72,-9.25,43626.0,R
2,2018-02-28,부국증권,KOSPI대비,증권대비,28500.0,14.95,6.45,32662.0,R
3,2018-03-30,부국증권,KOSPI대비,증권대비,27150.0,8.83,3.43,3351.0,R
4,2018-04-30,부국증권,KOSPI대비,증권대비,24350.0,-5.12,-12.07,4966.0,B
5,2018-05-31,부국증권,KOSPI대비,증권대비,23900.0,-3.17,-12.05,2859.0,B
6,2018-06-29,부국증권,KOSPI대비,증권대비,22400.0,-5.21,-8.27,1575.0,R
7,2018-07-31,부국증권,KOSPI대비,증권대비,23950.0,2.21,4.05,1120.0,R
8,2018-08-31,부국증권,KOSPI대비,증권대비,22850.0,-3.28,-2.25,1132.0,B
9,2018-09-28,부국증권,KOSPI대비,증권대비,22950.0,-3.71,-5.19,794.0,B


# 펀더멘탈

In [87]:
r = requests.get('https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=001270')

In [88]:
bs = BeautifulSoup(r.text, 'lxml')

In [91]:
r.text

'\r\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\r\n<html lang="ko">\r\n<head>\r\n\t<title>온라인기업정보 - 기업모니터 - 기업개요(부국증권)</title>\r\n\t<link rel="shortcut icon" href="/favicon.ico" />\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n\t<script language="javascript" src="/include/domain.js" type="text/javascript"></script>\r\n\t<!--[if (!IE) | (gt IE 8)]>\r\n    \t<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" />\r\n    <![endif]-->\r\n\t<!--[if IE 6]>\r\n            <link href="/include/only_ie6.css" type="text/css" rel="stylesheet"/>\r\n        <![endif]-->\r\n\t<!--[if lt IE 8]>\r\n            <link href="/include/lt_ie8.css" type="text/css" rel="stylesheet"/>\r\n        <![endif]-->\r\n\t<link href="/v2/include/css?v=q7Pq73b9jj00gmFmOVUSxzXv_5FtLNW9qxkqFwJqh-g1" rel="stylesheet"/>\r\n\r\n\t<link href="/v2/include/css1?v=2Kc54WAY0-YKwI0WYHBphRe4FmbyttZ0_0A_hA

In [98]:
fundamental_bs = bs.find_all("table",class_ = "gHead03").find_all("tr")

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [99]:
fundamental_bs

[<tr>
 <th style="padding-left:0; padding-right:0">주요지표</th>
 <th style="padding-left:0; padding-right:0">2018/12(A)</th>
 <th class="endLine" style="padding-left:0; padding-right:0">2019/12(E)</th>
 </tr>, <tr>
 <th class="left" scope="row">PER</th>
 <td class="num">10.14</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">PBR</th>
 <td class="num">0.42</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">PCR</th>
 <td class="num">611.73</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">EV/EBITDA</th>
 <td class="num">13.36</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">EPS</th>
 <td class="num">2,170원</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">BPS</th>
 <td class="num">51,862원</td>
 <td class="num noline-right"></td>
 </tr>, <tr>
 <th class="left" scope="row">EBITDA</th>
 <td class="num">390.8억원</td>
 <td class=

In [100]:
pd.read_html(str(bs.find("table", class_="gHead03")))[0]

,주요지표,2018/12(A),2019/12(E)
0,PER,10.14,NaN
1,PBR,0.42,NaN
2,PCR,611.73,NaN
3,EV/EBITDA,13.36,NaN
4,EPS,"2,170원",NaN
5,BPS,"51,862원",NaN
6,EBITDA,390.8억원,NaN
7,현금DPS,"1,200원",NaN
8,현금배당수익률,5.45%,NaN
9,회계기준,연결,연결


In [101]:
print('hello')

hello
